In [96]:
import cv2
import PIL
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import Video, widgets
from IPython.display import display

%matplotlib inline

In [97]:
if "google.colab" in str(get_ipython()):
    from google.colab.patches import cv2_imshow

    imshow = cv2_imshow
else:

    def imshow(img):
        img = img.clip(0, 255).astype("uint8")
        if img.ndim == 3:
            if img.shape[2] == 4:
                img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)
            else:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        display(PIL.Image.fromarray(img))

In [98]:
def resize_img(img, div=3):
    return cv2.resize(img, (img.shape[1]//div, img.shape[0]//div))

def save_first_frame(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Read the first frame
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        print("Error: Could not read the first frame.")
        return

    # Release the video capture object
    cap.release()

    return frame

def find_closest_points_to_corners(points, corners):
    closest_points = []
    for corner in corners:
        distances = np.linalg.norm(points - corner, axis=1)
        closest_point_index = np.argmin(distances)
        closest_points.append(points[closest_point_index])
    return np.array(closest_points)

def get_closest_corners(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    dst = cv2.cornerHarris(blurred, 5, 3, 0.02)
    dst = cv2.dilate(dst, None)
    corners = np.argwhere(dst > 0.01 * dst.max())

    # Find corners closest to image corners
    image_corners = np.array([[0, 0], [0, frame.shape[1]], [frame.shape[0], 0], [frame.shape[0], frame.shape[1]]])
    closest_corners = find_closest_points_to_corners(corners, image_corners)

    return closest_corners

def get_target_points(target_square_size=1000):
    target_points = np.array([[0, 0], [0, target_square_size - 1], [target_square_size - 1, 0], [target_square_size - 1, target_square_size - 1]], dtype=np.float32)
    return target_points

def detect_hand(frame, show_threshold=False):
    img_hls = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)

    # Define the skin color range in HLS
    skin_color_lower = np.array([0, 190, 100], dtype=np.uint8)
    skin_color_upper = np.array([15, 240, 255], dtype=np.uint8)

    # Create a mask using the skin color range
    range_mask = cv2.inRange(img_hls, skin_color_lower, skin_color_upper)

    # Remove noise with blurring
    blurred = cv2.blur(range_mask, (10, 10))

    # Threshold the blurred image to create a binary mask
    _, thresholded = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)

    roi1 = thresholded[0:img_hls.shape[0], 0:img_hls.shape[1]//2]
    roi2 = thresholded[0:int(img_hls.shape[0]/1.7), img_hls.shape[1]//2:img_hls.shape[1]]

    if show_threshold:
        cv2.imshow('Threshold hand', thresholded)

    if np.any(roi1 == 255) or np.any(roi2 == 255):
        return True
    
    return False

def get_squares_occupation(calibrated_image, grid_division, show_img=True):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.75
    thickness = 2

    square_size = grid_division[1] - grid_division[0]

    calibrated_gray = cv2.cvtColor(calibrated_image, cv2.COLOR_BGR2GRAY)
    calibrated_edges = cv2.Canny(calibrated_gray, 50, 150)

    space_img = calibrated_image.copy()

    empty_check_offset = 20
    kernel_size = square_size - empty_check_offset
    artifact_tolerance = 300
    col_imgs = []
    square_color = "Black"
    for x in grid_division[:-1]:
        row_imgs = []
        for y in grid_division[:-1]:
            x_d = x+empty_check_offset//2, x+empty_check_offset//2+kernel_size
            y_d = y+empty_check_offset//2, y+empty_check_offset//2+kernel_size

            edge_board_square = calibrated_edges[x_d[0]:x_d[1], y_d[0]:y_d[1]]
            edge_board_square_bin = np.where(edge_board_square==255, 1, edge_board_square)
            edge_board_square_value = np.sum(edge_board_square_bin)

            row_imgs.append(edge_board_square)
            # imshow(resize_img(edge_board_square, 1))

            if edge_board_square_value > artifact_tolerance:
                # cv2.putText(space_img, f"{square_color} sq", (y+20, x+50), font, font_scale, font_color, thickness)
                cv2.putText(space_img, f"Figure", (y+16, x+60), font, font_scale, (0, 0, 255), thickness)

            elif edge_board_square_value <= artifact_tolerance:
                # cv2.putText(space_img, f"{square_color} sq", (y+20, x+50), font, font_scale, font_color, thickness)
                cv2.putText(space_img, f"Empty", (y+16, x+60), font, font_scale, (0, 255, 0), thickness)
                
            square_color = "White" if square_color == "Black" else "Black"
                
        square_color = "White" if square_color == "Black" else "Black"

        # imshow(resize_img(np.concatenate(row_imgs, 1), 1))
        col_imgs.append(np.concatenate(row_imgs, 1))

    if show_img:
        cv2.imshow('Labelled_img', space_img)

    return space_img

def get_corner_diff(ref_corners, new_corners):
    ref_corners = np.array(ref_corners)
    new_corners = np.array(new_corners)

    distances = np.linalg.norm(ref_corners - new_corners, axis=1)
    
    return distances

def correct_corners(corners, ref_corners, marked_change):
    updated_corners = corners.copy()
    good_corners = [(corner, id) for id, corner in enumerate(corners) if not marked_change[id]]
    bad_corners = [(corner, id) for id, corner in enumerate(corners) if marked_change[id]]

    num_incorrect = len(bad_corners)
    
    # print(corners)
    if num_incorrect == 0:
        return corners
    if num_incorrect == 4:
        return ref_corners
    
    good_corner, g_id = good_corners[-1]
    
    for bad_corner, b_id in bad_corners:
        delta_x = ref_corners[b_id][0] - ref_corners[g_id][0]
        delta_y = ref_corners[b_id][1] - ref_corners[g_id][1]

        # Update the position of the faulty corner based on the difference
        updated_corners[b_id][0] = good_corner[0] + delta_x
        updated_corners[b_id][1] = good_corner[1] + delta_y
        
        # print(bad_corner, updated_corners[b_id][0], updated_corners[b_id][1])

    return updated_corners

Keypoint Detection

In [99]:
def is_point_within_grid(keypoint, grid):
    x, y = keypoint.pt
    return (x > grid[0]) and (x < grid[1]) and (y > grid[2]) and (y < grid[3])

In [100]:
def detect_pawns(frame, grid_division, good_matches, reference_keypoints):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.75
    thickness = 2
    matched_keypoints_indices = [match.queryIdx for match in good_matches]

    for i in range(len(grid_division[:-1])):
        for j in range(len(grid_division[:-1])):
            grid_bounds = [grid_division[i], grid_division[i + 1], grid_division[j], grid_division[j + 1]]
            is_pawn_present = any(
                is_point_within_grid(reference_keypoints[matched_keypoint], grid_bounds)
                for matched_keypoint in matched_keypoints_indices
            )
            
            label = "Pawn" if is_pawn_present else "Empty"
            color = (0, 0, 255) if is_pawn_present else (0, 255, 0)
            cv2.putText(frame, label, (grid_division[i] + 16, grid_division[j] + 60), font, font_scale, color, thickness)

    return frame

In [101]:
'Chess_Data\\chess_hard1.mp4'
video_capture = cv2.VideoCapture('Chess_Data\\chess_hard1.mp4')

target_square_size = 1000  # Adjust the size as needed
target_points = np.array([[0, 0], [0, target_square_size - 1], [target_square_size - 1, 0], [target_square_size - 1, target_square_size - 1]], dtype=np.float32)

is_first_frame = True
distance_threshold = 7
reference_path = 'Chess_Data\\pawn.jpg'
reference_img = cv2.imread(reference_path)
reference_gray = cv2.cvtColor(reference_img, cv2.COLOR_BGR2GRAY)

secondery_reference = 'Chess_Data\\pawn2.jpg'
reference_img2 = cv2.imread(secondery_reference)
reference_gray2 = cv2.cvtColor(reference_img2, cv2.COLOR_BGR2GRAY)

sift = cv2.SIFT_create()
reference_keypoints, reference_descriptors = sift.detectAndCompute(reference_gray, None)
reference_keypoints2, reference_descriptors2 = sift.detectAndCompute(reference_gray2, None)
bf = cv2.BFMatcher()

while True:
    ret, frame = video_capture.read()
    if not ret:
        break
    
    closest_corners = get_closest_corners(frame)

    if is_first_frame:
        ref_corners = closest_corners.copy()
        is_first_frame = False

    # If change in corners is big, use last accepted reference corners
    distances = get_corner_diff(ref_corners, closest_corners)

    marked_change = [False, False, False, False]
    for c in range(4):
        if distances[c] > distance_threshold:
            marked_change[c] = True

    corrected_corners = correct_corners(closest_corners, ref_corners, marked_change)
    ref_corners = corrected_corners.copy()

    homography_matrix, _ = cv2.findHomography(corrected_corners[:,::-1], target_points)

    # Apply the perspective transformation
    calibrated_image = cv2.warpPerspective(frame, homography_matrix, (target_square_size, target_square_size))

    grid_img = calibrated_image.copy()
    offset = 59 # Board edge offset
    grid_division = np.linspace(offset, target_square_size-offset, 9, dtype=int)
    for x in grid_division:
        cv2.line(grid_img, (x, 0+offset), (x, target_square_size-offset), color=(0, 0, 255), thickness=2)  
        cv2.line(grid_img, (0+offset, x), (target_square_size-offset, x), color=(0, 0, 255), thickness=2)  
        
    calibrated_frame = calibrated_image.copy()
    calibrated_gray = cv2.cvtColor(calibrated_frame, cv2.COLOR_BGR2GRAY)
    frame_keypoints, frame_descriptors = sift.detectAndCompute(calibrated_gray, None)

    # Match keypoints between the reference image and the current frame
    matches = bf.knnMatch(reference_descriptors, frame_descriptors, k=2)
    matches2 = bf.knnMatch(reference_descriptors2, frame_descriptors, k=2)

    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.9*n.distance:
            good_matches.append(m)

    good_matches2 = []
    for m, n in matches2:
        if m.distance < 0.9*n.distance:
            good_matches2.append(m)
        
    total_matches = []
    for m in good_matches:
        total_matches.append(m)
    for m in good_matches2:
        total_matches.append(m)
            
    calibrated_frame = detect_pawns(calibrated_frame, grid_division, total_matches, frame_keypoints)
    # Draw matches on the current frame
    img_matches = cv2.drawMatches(reference_img, reference_keypoints, calibrated_frame, frame_keypoints, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    img_matches = cv2.drawMatches(reference_img2, reference_keypoints2, img_matches, frame_keypoints, good_matches2, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    

    # Display the current frame with matches
    cv2.imshow('Matches', img_matches)
    

    # Display the result    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

In [103]:
##TODO: encode things manually and possibly fix this